<a href="https://colab.research.google.com/github/PraveenKumar-pk-star/GenAI_RAG/blob/main/Rag_with_sql_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain==0.3.27
!pip install langchain-core==0.3.79
!pip install langchain-openai
!pip install chromadb
# ! pip install pypdf
!pip install langchain-community==0.3.31
!pip install langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.1 MB/s e

In [2]:
! pip uninstall -y langchain-core
! pip uninstall -y langchain-openai

Found existing installation: langchain-core 1.0.2
Uninstalling langchain-core-1.0.2:
  Successfully uninstalled langchain-core-1.0.2
Found existing installation: langchain-openai 1.0.1
Uninstalling langchain-openai-1.0.1:
  Successfully uninstalled langchain-openai-1.0.1


In [3]:
! pip install langchain-core==0.3.79
! pip install langchain-openai==0.3.35


  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.5 MB/s eta 0:00:00


In [17]:
 ! pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 8.1 MB/s eta 0:00:00


In [24]:
from getpass import getpass
from google.colab import files
import os
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community .vectorstores import Chroma
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts import PromptTemplate
from langchain.memory import SQLChatMessageHistory


In [30]:
Open_Api_key=getpass()

··········


In [7]:
upload_file=files.upload()

Saving earth_full_pdf.pdf to earth_full_pdf.pdf


In [8]:
import os

for root, dirs, files in os.walk("/content"):
    for file in files:
        print(os.path.join(root, file))

/content/earth_full_pdf.pdf
/content/.config/.last_opt_in_prompt.yaml
/content/.config/default_configs.db
/content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
/content/.config/gce
/content/.config/config_sentinel
/content/.config/active_config
/content/.config/.last_survey_prompt.yaml
/content/.config/.last_update_check.json
/content/.config/logs/2025.10.30/13.36.07.857469.log
/content/.config/logs/2025.10.30/13.35.52.212263.log
/content/.config/logs/2025.10.30/13.35.58.184082.log
/content/.config/logs/2025.10.30/13.35.10.281184.log
/content/.config/logs/2025.10.30/13.36.08.663879.log
/content/.config/logs/2025.10.30/13.35.42.751639.log
/content/.config/configurations/config_default
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/california_housing_train.csv
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/mnist_train_small.csv


In [15]:
text_path=""
def extract_text(text_path):
  doc=PyPDFLoader(text_path)
  pages=doc.load()
  return pages

In [19]:
pdf_path="/content/earth_full_pdf.pdf"
text_doc=extract_text(pdf_path)

In [34]:
session_id="praveen_id:1"
memory=SQLChatMessageHistory(session_id=session_id,connection_string="sqlite:///chat_history.db")



In [39]:
doc_spliter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
doc_emmbeding=OpenAIEmbeddings(api_key=Open_Api_key)
doc_vectdb=Chroma.from_documents(documents=text_doc,embedding=doc_emmbeding,persist_directory="text_chromadb")
ret=doc_vectdb.as_retriever()


In [40]:
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0,api_key=Open_Api_key)

In [52]:
prompt_qa="""You are an intelligent assistant. Use the following context and chat history
to answer the question clearly and correctly
If there is no answer in the context, say: "I don't know. The information is not provided in the document."

Context:
{context}

Chat History:
{chat_history}

Question: {input}

Answer:
"""
prompt=PromptTemplate(template=prompt_qa,input_variables=["context","chat_history","input"])



In [57]:
question_answer_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)
retraival_chain=create_retrieval_chain(
    retriever=ret,
    combine_docs_chain=question_answer_chain
)

In [58]:
while True:
    user_input = input(" You: ")
    if user_input.lower() in ["exit", "quit"]:
        print(" AI: Goodbye!")
        break

    # Save user message
    memory.add_user_message(user_input)

    # Prepare chat history
    chat_history_text = "\n".join(
        [f"User: {m.content}" if m.type == "human" else f"AI: {m.content}" for m in memory.messages]
    )

    # Run the chain
    result = retraival_chain.invoke({
        "input": user_input,
        "chat_history": chat_history_text
    })
        # Extract the answer
    answer = result.get("answer", result.get("output_text", ""))

    # Save AI response
    memory.add_ai_message(answer)

    print(f"AI: {answer}\n")

 You: what  is document about
AI: The document is a sample chapter from the Encyclopedia of Life Support Systems (EOLSS) titled "Earth as a Planet" by Anthony W. Brian. It provides an overview of Earth, discussing its origin, interior structure, plate tectonics, geoid, atmosphere, and magnetic field. It also covers the Moon, including its status as Earth's only satellite, lunar missions, and its origin. The chapter emphasizes Earth's unique position in the solar system as the only planet known to support life, highlighting its dynamic systems and the significance of recent space exploration in enhancing our understanding of global geology.

 You: what population of the world
AI: I don't know. The information is not provided in the document.

 You: who is prime minister of indoa
AI: I don't know. The information is not provided in the document.

 You: who is prime minister of india
AI: As of my last knowledge update in October 2023, the Prime Minister of India is Narendra Modi. He has b